<a href="https://colab.research.google.com/github/Legalworkflow/Extract_-_Redact-PDF/blob/master/PDF_Compliance_Tool_BATCH_Hocr_%2B_Redaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔒 PDF HOCR + Redact Preserve layout with HOCR and Tesseract5
This notebook can function in a few ways, depending on how you set the run_ocr and redact_pdf variables: It prompts you to mount Google Drive. You can change input/output paths or just create a "Input" & "Output" folder in MyDrive to re use.

Once Mounted it installs all pkgs and dependencies per cell. So if you need OCR but not Redaction, set the **redact_pdf** Variable to "False" and you don't install the  unnecessary redaction pkgs. Same with Redaction on Text based PDFs, there's no need for OCR: Set to "False" and you can simply enter your word list in the "", "", format and go straight to redaction.

# 👨‍💻 0
OR DO IT ALL TRUE+TRUE = OCR/Redaction Both.

In [ ]:
# 📦 Install PyMuPDF
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 35.0 MB/s eta 0:00:00


In [ ]:
# 📂 Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 🧾 Word List
words_to_redact = ["1628", "High", "Cir.", "1273", "1500 Fawn Run Crossing", "Fawn Run Crossing", "Fawn Run",
    "Fawn", "Run", "crossing", "Skyridge", "Dallas", "Cheyenne", "Austin", "Norman", "Tucson",
    "Yukon", "Cleveland", "Tiffaney Norton", "Dallas Norton", "Christopher", "Brianna Jackson",
    "Brionna Jackson", "Brianna", "Bri", "Jeffry Jackson", "Jeffrey Jackson", "Caitlin Jackson",
    "Christi Cornett", "Christi Comett", "Angela", "Thagard", "Marilyn", "Williams", "Eufaula",
    "Jones", "73069", "73071", "Cory", "Lori", "Puckett", "Virgil", "Black", "Ortega", "Jackson",
    "Pierce", "Troy", "Judy", "Sean", "Bailey", "Ferguson", "Norton", "Whatley", "Nedwick",
    "Douglas", "Balkman", "Tayra", "Christy", "jilge", "Mary", "Abbott", "Children's", "Christi",
    "Christy", "Jennifer", "Shyanne", "Tara", "Riley", "Ryleigh", "Caitlyn", "Caitlin", "B.J.",
    "B. J.", "BJ", "Bo", "Bobo", "Tay", "Tiffany", "Cornett", "Jacobi", "Norman", "Nonnan",
    "Dallas C. Norton", "Ortega", "MINOR CHILD B.J.", "bri", "Eisenhower", "Long Fellow",
    "Norman North", "Angela","Thagard","Tanya", "Burcham", "Thomas", "Keith", "john","Hadden",
    "127", "Crestland", "Charles", "Peters", "200", "Eufaula", "Jones", "MR. ORTEGA"]

In [ ]:
# 📌 Optional: OCR with Tesseract 5 + HOCR (accurate overlay, in-place update)
run_ocr = True  # 🔁 Set to False to disable

if run_ocr == True:
    # Install system dependencies for pdf2image
    !apt-get update
    !apt-get install -y poppler-utils

    !pip install pytesseract
    import pytesseract
    !pip install pdf2image
    from pdf2image import convert_from_path
    from PIL import Image
    import os
    !pip install PyPDF2
    from PyPDF2 import PdfMerger
    # !pip install poppler # This is a system dependency, not a pip package

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4,932 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,148 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-secur

In [ ]:
input_dir = '/content/drive/MyDrive/Input'
def ocr_pdf_hocr (input_path, output_path):
    import pytesseract
    temp_img_dir = '/content/ocr_images'
    os.makedirs(temp_img_dir, exist_ok=True)
    images = convert_from_path(input_path, dpi=300, fmt='jpeg', output_folder=temp_img_dir)
    page_paths = []

    for i, img in enumerate(images):
        img_path = f'/content/page_{i}.jpeg'
        img.save(img_path, 'JPEG')

        # Generate HOCR (layout-aware text layer)
        hocr_output = pytesseract.image_to_pdf_or_hocr(img, extension='pdf', lang='eng', config='hocr')
        pdf_out = f'/content/page_hocr_{i}.pdf'
        with open(pdf_out, 'wb') as f:
            f.write(hocr_output)
        page_paths.append(pdf_out)

    merger = PdfMerger()
    for p in page_paths:
        merger.append(p)
    merger.write(output_path)
    merger.close()
    print(f"✅ HOCR overlay OCR complete: {output_path}")

# 🔁 Overwrite files in-place
for file in os.listdir(input_dir):
    if file.lower().endswith('.pdf'):
        src = os.path.join(input_dir, file)
        temp_out = os.path.join('/content', f'ocr_{file}')
        try:
            ocr_pdf_hocr(src, temp_out)
            import shutil
            shutil.move(temp_out, src) # 🔁 Replace original with OCR’d version
        except Exception as e:
            print(f"⚠️ HOCR OCR failed for {file}: {e}")

In [ ]:
# 🧼 Optional Redaction Script
redact_pdf = True 🔁 Set to False to disable

If redact_pdf == True:
import fitz
import os
import re # Import regular expressions - Keep import just in case, though not used in new redact_pdf

input_dir = "/content/drive/MyDrive/Input" # Corrected directory name
output_dir = "/content/drive/MyDrive/Output"
os.makedirs(output_dir, exist_ok=True)

def redact_pdf(input_path, output_path, words):
    doc = fitz.open(input_path)
    word_set = set(w.lower() for w in words)  # Lowercase for case-insensitive match

    for page in doc:
        wordlist = page.get_text("words")  # list of (x0, y0, x1, y1, "word", block_no, line_no, word_no)
        for w in wordlist:
            if w[4].lower() in word_set:
                rect = fitz.Rect(w[:4])
                page.draw_rect(rect, fill=(0, 0, 0))
    doc.save(output_path, garbage=4, deflate=True)
    doc.close()


# 🔄 Process All PDFs
for file in os.listdir(input_dir):
    if file.lower().endswith(".pdf"):
        input_path = os.path.join(input_dir, file)
        output_path = os.path.join(output_dir, f"redacted_{file}")
        print(f"Redacting: {file}")
        redact_pdf(input_path, output_path, words_to_redact)
        print(f"Saved to: {output_path}")

print("✅ Redaction complete.")